In [1]:
!pip install transformers datasets evaluate pyspellchecker 
!pip install --upgrade accelerate
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

# Подготовка

In [2]:
# to hide warnings
import warnings
warnings.filterwarnings('ignore')

# basic data processing
import os
import datetime
import pandas as pd
import numpy as np

# progress bar
from tqdm.auto import tqdm
from tqdm import tqdm_notebook
tqdm.pandas()

# for wordcloud
from PIL import Image
from wordcloud import WordCloud

# for aesthetics and plots
from IPython.display import display, Markdown, clear_output
from termcolor import colored

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "notebook"

# for model
import tensorflow as tf
import tensorflow_hub as hub
import keras.layers as layers
from keras.models import Model
from keras import backend as K
import keras
from keras.models import load_model

from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoConfig, AutoModel, AutoModelForSequenceClassification, AutoTokenizer
import torch
import torch.nn as nn

from sklearn.metrics import accuracy_score
from transformers import pipeline
from torch.utils.data import DataLoader, TensorDataset
from transformers import TrainingArguments, Trainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
import nltk
from nltk.corpus import stopwords, wordnet
from spacy.cli import download
from spacy import load
import re
import nltk 
from nltk.stem.wordnet import WordNetLemmatizer
from spellchecker import SpellChecker

nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('wordnet2022')

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet # temp fix for lookup error.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package wordnet2022 to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet2022.zip.


cp: cannot stat '/usr/share/nltk_data/corpora/wordnet2022': No such file or directory


# Токенайзер

In [4]:
model_name = "finiteautomata/bertweet-base-sentiment-analysis"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenizing(examples):
    # return tokenizer.encode_plus(
    #     examples,
    #     add_special_tokens=True,
    #     max_length=256,
    #     padding='max_length',
    #     truncation=True,
    #     return_attention_mask=True,
    #     return_tensors='pt'
    # )

   return tokenizer(examples["text"],
                    add_special_tokens=True,
                    max_length=50,
                    padding='max_length',
                    truncation=True,
                    return_attention_mask=True,
                    return_tensors='pt')


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


# Скачивание датасета и препроцессинг

In [5]:
 !pip install -q kaggle

In [6]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [7]:
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list


ref                                                       title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
arnabchaki/data-science-salaries-2023                     Data Science Salaries 2023 💸                       25KB  2023-04-13 09:55:16          15912        432  1.0              
chitrakumari25/corona-virus-latest-data-2023              Corona virus latest data 2023                      10KB  2023-04-29 16:00:51           1582         40  1.0              
utkarshx27/starbucks-nutrition                            Starbucks Nutrition Facts                           2KB  2023-05-10 05:42:59           1027         36  1.0              
utkarshx27/monthly-transportation-statistics              Monthly Transportation Statistics         

In [8]:
# set of stop words declared
stop_words = stopwords.words('english')

updated_stop_words = stop_words.copy()
for word in stop_words:
    if "n't" in word or "no" in word or word.endswith('dn') or word.endswith('sn') or word.endswith('tn'):
        updated_stop_words.remove(word)

# custom select words you don't want to eliminate
words_to_remove = ['for','by','with','against','shan','don','aren','haven','weren','until','ain','but','off','out']
for word in words_to_remove:
    updated_stop_words.remove(word)

In [9]:
# Defining dictionary containing all emojis with their meanings.
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

# Defining regex patterns.
urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
userPattern       = '@[^\s]+'
alphaPattern      = "[^a-zA-Z0-9]"
sequencePattern   = r"(.)\1\1+"
seqReplacePattern = r"\1\1"

# creating instance of spellchecker
spell = SpellChecker()

# creating instance of lemmatizer
lemm = WordNetLemmatizer()


def preprocess(tweet):
    # lowercase the tweets
    tweet = tweet.lower().strip()
    
    # REMOVE all URls
    tweet = re.sub(urlPattern,'',tweet)
    
    # Replace all emojis.
    for emoji in emojis.keys():
        tweet = tweet.replace(emoji, "emoji" + emojis[emoji])        
    
    # Remove @USERNAME
    tweet = re.sub(userPattern,'', tweet)        
    
    # Replace all non alphabets.
    tweet = re.sub(alphaPattern, " ", tweet)
    
    # Replace 3 or more consecutive letters by 2 letter.
    tweet = re.sub(sequencePattern, seqReplacePattern, tweet)

    splitted_tweet = tweet.split()
    # spell checks
#     misspelled = spell.unknown(splitted_tweet)
#     if misspelled == set():
#         pass
#     else:
#         for i,word in enumerate(misspelled):
#             splitted_tweet[i] = spell.correction(word)

    tweetwords = ''
    for word in splitted_tweet:
        # Checking if the word is a stopword.
        if word not in updated_stop_words:
            if len(word)>1:
                # Lemmatizing the word.
                lem_word = lemm.lemmatize(word)
                tweetwords += (lem_word+' ')
    
    return tweetwords


In [10]:
! kaggle datasets download -d 'kazanova/sentiment140'
! kaggle datasets download -d 'ankurzing/sentiment-analysis-for-financial-news'

!unzip sentiment140.zip
!unzip sentiment-analysis-for-financial-news.zip

 80% 65.0M/80.9M [00:00<00:00, 114MB/s]
100% 80.9M/80.9M [00:00<00:00, 127MB/s]
  0% 0.00/903k [00:00<?, ?B/s]
100% 903k/903k [00:00<00:00, 121MB/s]
Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  
Archive:  sentiment-analysis-for-financial-news.zip
  inflating: FinancialPhraseBank/License.txt  
  inflating: FinancialPhraseBank/README.txt  
  inflating: FinancialPhraseBank/Sentences_50Agree.txt  
  inflating: FinancialPhraseBank/Sentences_66Agree.txt  
  inflating: FinancialPhraseBank/Sentences_75Agree.txt  
  inflating: FinancialPhraseBank/Sentences_AllAgree.txt  
  inflating: all-data.csv            


In [11]:
fin_df = pd.read_csv('all-data.csv', delimiter=',', encoding='latin-1', header=None)
fin_df.columns = ['label', 'text']

fin_df.loc[fin_df['label'] == 'negative', 'label'] = 0
fin_df.loc[fin_df['label'] == 'positive', 'label'] = 1
fin_df.loc[fin_df['label'] == 'neutral', 'label'] = 2

findf_train, findf_test = train_test_split(fin_df, test_size=0.7, random_state=42) #поменять потом

train_fin = Dataset.from_pandas(findf_train).shuffle(seed=55).map(tokenizing, batched=True)
test_fin = Dataset.from_pandas(findf_test).shuffle(seed=55).map(tokenizing, batched=True)

fin_df['text'] = fin_df['text'].apply(lambda x: preprocess(x))
tokenized_fin = Dataset.from_pandas(fin_df).shuffle(seed=55).map(tokenizing, batched=True)

Map:   0%|          | 0/1453 [00:00<?, ? examples/s]

Map:   0%|          | 0/3393 [00:00<?, ? examples/s]

Map:   0%|          | 0/4846 [00:00<?, ? examples/s]

In [12]:
col_names =  ['label', 'id', 'date', 'flag','user','text']
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding = "ISO-8859-1", names=col_names)
df.drop(columns=['id', 'date', 'flag','user'], inplace=True)
df, _ = train_test_split(df, test_size=0.8, random_state=42)

df['text'] = df['text'].apply(lambda x: preprocess(x))

df.loc[df['label'] == 1, 'label'] = 0
df.loc[df['label'] == 4, 'label'] = 1

df_test_val, df_train = train_test_split(df, test_size=0.7, random_state=42)
df_test, df_val = train_test_split(df_test_val, test_size=0.5, random_state=42)

train_set = Dataset.from_pandas(df_train).shuffle(seed=55)
test_set = Dataset.from_pandas(df_test).shuffle(seed=55)
val_set = Dataset.from_pandas(df_val).shuffle(seed=55)

tokenized_train = train_set.map(tokenizing, batched=True)
tokenized_test = test_set.map(tokenizing, batched=True)
tokenized_val = val_set.map(tokenizing, batched=True)

Map:   0%|          | 0/224000 [00:00<?, ? examples/s]

Map:   0%|          | 0/48000 [00:00<?, ? examples/s]

Map:   0%|          | 0/48000 [00:00<?, ? examples/s]

In [13]:
print(len(train_set), len(test_set), len(val_set))

224000 48000 48000


In [14]:
# fin_df = pd.read_csv('all-data.csv', delimiter=',', encoding='latin-1', header=None)
# fin_df.columns = ['label', 'text']

# fin_df.loc[fin_df['label'] == 'negative', 'label'] = 0
# fin_df.loc[fin_df['label'] == 'positive', 'label'] = 1
# fin_df.loc[fin_df['label'] == 'neutral', 'label'] = 2

# fin_df['text'] = fin_df['text'].apply(lambda x: preprocess(x))
# fin_df['tokenized'] = fin_df['text'].apply(tokenizing)

# fin_train, fin_test = train_test_split(fin_df, test_size=0.2, random_state=42)
# fin_train = fin_train.reset_index(drop=True)
# fin_test = fin_test.reset_index(drop=True)

# # Преобразование токенизированных данных в тензоры
# train_texts = torch.cat([fin_train['tokenized'][i]['input_ids'] for i in range(len(fin_train))], dim=0)
# train_masks = torch.cat([fin_train['tokenized'][i]['attention_mask'] for i in range(len(fin_train))], dim=0)
# train_token = torch.cat([fin_train['tokenized'][i]['token_type_ids'] for i in range(len(fin_train))], dim=0)
# fin_train['label'] = fin_train['label'].astype(int)
# train_labels = torch.tensor(fin_train['label'].values)

# test_texts = torch.cat([fin_test['tokenized'][i]['input_ids'] for i in range(len(fin_test))], dim=0)
# test_masks = torch.cat([fin_test['tokenized'][i]['attention_mask'] for i in range(len(fin_test))], dim=0)
# test_token = torch.cat([fin_test['tokenized'][i]['token_type_ids'] for i in range(len(fin_test))], dim=0)
# fin_test['label'] = fin_test['label'].astype(int)
# test_labels = torch.tensor(fin_test['label'].values)

# # Создание DataLoader для тренировочного набора данных
# fin_train_dataset = TensorDataset(train_texts, train_masks, train_token, train_labels)
# fin_train_dataloader = DataLoader(fin_train_dataset, batch_size=16, shuffle=True)

# # Создание DataLoader для тестового набора данных
# fin_test_dataset = TensorDataset(test_texts, test_masks, test_token, test_labels)
# fin_test_dataloader = DataLoader(fin_test_dataset, batch_size=16, shuffle=True)


In [15]:
# col_names =  ['label', 'id', 'date', 'flag','user','text']
# df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding = "ISO-8859-1", names=col_names)
# df.drop(columns=['id', 'date', 'flag','user'], inplace=True)
# df, _ = train_test_split(df, test_size=0.75, random_state=42)

# df.loc[df['label'] == 1, 'label'] = 0
# df.loc[df['label'] == 4, 'label'] = 1

# df['text'] = df['text'].apply(lambda x: preprocess(x))
# df['tokenized'] = df['text'].apply(tokenizing)

# test_val_data, train_data = train_test_split(df, test_size=0.7, random_state=42)
# val_data, test_data = train_test_split(test_val_data, test_size=0.5, random_state=42)
# train_data = train_data.reset_index(drop=True)
# test_data = test_data.reset_index(drop=True)
# val_data = val_data.reset_index(drop=True)

# # Преобразование токенизированных данных в тензоры
# train_texts = torch.cat([train_data['tokenized'][i]['input_ids'] for i in range(len(train_data))], dim=0)
# train_masks = torch.cat([train_data['tokenized'][i]['attention_mask'] for i in range(len(train_data))], dim=0)
# train_token = torch.cat([train_data['tokenized'][i]['token_type_ids'] for i in range(len(train_data))], dim=0)
# train_data['label'] = train_data['label'].astype(int)
# train_labels = torch.tensor(train_data['label'].values)

# test_texts = torch.cat([test_data['tokenized'][i]['input_ids'] for i in range(len(test_data))], dim=0)
# test_masks = torch.cat([test_data['tokenized'][i]['attention_mask'] for i in range(len(test_data))], dim=0)
# test_token = torch.cat([test_data['tokenized'][i]['token_type_ids'] for i in range(len(test_data))], dim=0)
# test_data['label'] = test_data['label'].astype(int)
# test_labels = torch.tensor(test_data['label'].values)

# val_texts = torch.cat([val_data['tokenized'][i]['input_ids'] for i in range(len(val_data))], dim=0)
# val_masks = torch.cat([val_data['tokenized'][i]['attention_mask'] for i in range(len(val_data))], dim=0)
# val_token = torch.cat([val_data['tokenized'][i]['token_type_ids'] for i in range(len(val_data))], dim=0)
# val_data['label'] = val_data['label'].astype(int)
# val_labels = torch.tensor(val_data['label'].values)

# batch_size = 8
# # Создание DataLoader для тренировочного набора данных
# train_dataset = TensorDataset(train_texts, train_masks, train_token, train_labels)
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# # Создание DataLoader для тестового набора данных
# test_dataset = TensorDataset(test_texts, test_masks, test_token, test_labels)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# # Создание DataLoader для валидационного набора данных
# val_dataset = TensorDataset(val_texts, val_masks, val_token, val_labels)
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


# Zero-shot classification (baseline)

In [26]:
print(fin_df[fin_df['label'] == 0].shape[0],
      fin_df[fin_df['label'] == 1].shape[0],
      fin_df[fin_df['label'] == 2].shape[0])

noneu_df = fin_df[fin_df['label'] != 2]

604 1363 2879


In [27]:
all_ans = []

In [28]:
from sklearn.metrics import f1_score

Эксперимент с дефолтными названиями классов

In [29]:
label2id = {"negative":0, "positive":1, "neutral":2}
labels = list(label2id.keys())

classifier = pipeline("zero-shot-classification", device=0)

ans = classifier(fin_df['text'].tolist(), labels)

pred_labels = []
for x in ans:
    pred_labels.append(x['labels'][0])
pred_id = [label2id[x] for x in pred_labels]
all_ans.append(pred_id)

print(accuracy_score(pred_id, fin_df['label'].tolist()))
print(f1_score(fin_df['label'].tolist(), pred_id, average='weighted'))

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


0.3926950061906727
0.2744397666878865


In [ ]:
# classifier = pipeline("zero-shot-classification", device=0)
# labels = ["negative", "positive"]
# label2id = {labels[0]:0, labels[1]:1}

# ans = classifier(noneu_df['text'].tolist(), labels)

# pred_labels = []
# for x in ans:
#     pred_labels.append(x['labels'][0])
# pred_id = [label2id[x] for x in pred_labels]

# all_ans.append(pred_id)
# print(accuracy_score(pred_id, noneu_df['label'].tolist()))

Эксперимент с "новостными" классами

In [30]:
classifier = pipeline("zero-shot-classification", device=0)
labels = ["negative news", "positive news", "neutral news"]
label2id = {labels[0]:0, labels[1]:1, labels[2]:2}

ans = classifier(fin_df['text'].tolist(), labels)

pred_labels = []
for x in ans:
    pred_labels.append(x['labels'][0])
pred_id = [label2id[x] for x in pred_labels]

all_ans.append(pred_id)
print(accuracy_score(pred_id, fin_df['label'].tolist()))
print(f1_score(fin_df['label'].tolist(), pred_id, average='weighted'))

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


0.4830788278992984
0.43257936407955166


In [ ]:
# classifier = pipeline("zero-shot-classification", device=0)
# labels = ["bad news", "good news"]
# label2id = {labels[0]:0, labels[1]:1}

# ans = classifier(noneu_df['text'].tolist(), labels)

# pred_labels = []
# for x in ans:
#     pred_labels.append(x['labels'][0])
# pred_id = [label2id[x] for x in pred_labels]

# all_ans.append(pred_id)
# print(accuracy_score(pred_id, noneu_df['label'].tolist()))

Эксперимент с нейтральный -> базовый

In [34]:
classifier = pipeline("zero-shot-classification", device=0)
labels = ["negative news", "positive news", "basic news"]
label2id = {labels[0]:0, labels[1]:1, labels[2]:2}

ans = classifier(fin_df['text'].tolist(), labels)

pred_labels = []
for x in ans:
    pred_labels.append(x['labels'][0])
pred_id = [label2id[x] for x in pred_labels]

all_ans.append(pred_id)
print(accuracy_score(pred_id, fin_df['label'].tolist()))
print(f1_score(fin_df['label'].tolist(), pred_id, average='weighted'))

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


0.49112670243499795
0.45161757165066885


In [33]:
classifier = pipeline("zero-shot-classification", device=0)
labels = ["bad news", "good news", "basic news"]
label2id = {labels[0]:0, labels[1]:1, labels[2]:2}

ans = classifier(fin_df['text'].tolist(), labels)

pred_labels = []
for x in ans:
    pred_labels.append(x['labels'][0])
pred_id = [label2id[x] for x in pred_labels]

all_ans.append(pred_id)
print(accuracy_score(pred_id, fin_df['label'].tolist()))
print(f1_score(fin_df['label'].tolist(), pred_id, average='weighted'))

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


0.481634337598019
0.43990089429005574


In [ ]:
classifier = pipeline("zero-shot-classification", device=0)
labels = ["loss", "income", "no impact"]
label2id = {labels[0]:0, labels[1]:1, labels[2]:2}

ans = classifier(fin_df['text'].tolist(), labels)

pred_labels = []
for x in ans:
    pred_labels.append(x['labels'][0])
pred_id = [label2id[x] for x in pred_labels]

all_ans.append(pred_id)
print(accuracy_score(pred_id, fin_df['label'].tolist()))

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


0.34234420140321914


In [ ]:
classifier = pipeline("zero-shot-classification", device=0)
labels = ["bad news", "good news", "no impact"]
label2id = {labels[0]:0, labels[1]:1, labels[2]:2}

ans = classifier(fin_df['text'].tolist(), labels)

pred_labels = []
for x in ans:
    pred_labels.append(x['labels'][0])
pred_id = [label2id[x] for x in pred_labels]

all_ans.append(pred_id)
print(accuracy_score(pred_id, fin_df['label'].tolist()))

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


0.387536112257532


# Sentiment-analysis pipeline

In [36]:
classifier = pipeline("sentiment-analysis", model=model_name, device=0)
sequences = fin_df['text'].tolist()
ans = classifier(sequences)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [37]:
ans_label = []
for x in ans:
    if x['label'] == 'NEG':
        ans_label.append(0)
    elif x['label'] == 'POS':
        ans_label.append(1)
    elif x['label'] == 'NEU':
        ans_label.append(2)


In [38]:
print(accuracy_score(ans_label, fin_df['label'].tolist()))
print(f1_score(fin_df['label'].tolist(), ans_label, average='weighted'))

0.6789104416013206
0.6455105858957492


In [39]:
classifier = pipeline("sentiment-analysis", model='yiyanghkust/finbert-tone', device=0)
sequences = fin_df['text'].tolist()
ans = classifier(sequences)

ans_label = []
for x in ans:
    if x['label'] == 'Negative':
        ans_label.append(0)
    elif x['label'] == 'Positive':
        ans_label.append(1)
    elif x['label'] == 'Neutral':
        ans_label.append(2)

print(accuracy_score(ans_label, fin_df['label'].tolist()))
print(f1_score(fin_df['label'].tolist(), ans_label, average='weighted'))

0.7162608336772596
0.6935253925714941


In [40]:
classifier = pipeline("sentiment-analysis", model='ProsusAI/finbert', device=0)
sequences = fin_df['text'].tolist()
ans = classifier(sequences)

In [41]:
ans_label = []
for x in ans:
    if x['label'] == 'negative':
        ans_label.append(0)
    elif x['label'] == 'positive':
        ans_label.append(1)
    elif x['label'] == 'neutral':
        ans_label.append(2)

print(accuracy_score(ans_label, fin_df['label'].tolist()))
print(f1_score(fin_df['label'].tolist(), ans_label, average='weighted'))

0.7975650020635576
0.7917259138497459


# Обучаем 

In [20]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

import numpy as np
from datasets import load_metric
from sklearn.metrics import accuracy_score


def compute_metrics(eval_pred):
    
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels, average='weighted')["f1"]
    return {"accuracy": accuracy, "f1": f1}

Обучим не предобученную модель на датасете твиттера

In [17]:
config = AutoConfig.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)
model =  AutoModelForSequenceClassification.from_config(config);

In [18]:
# model_path = '/content/drive/MyDrive/diploma_model'
# model = AutoModelForSequenceClassification.from_pretrained(model_path)

# config = AutoConfig.from_pretrained(model_path, num_labels=2, ignore_mismatched_sizes=True)
# model =  AutoModelForSequenceClassification.from_config(config);

In [18]:
repo_name = "learn_tweets"

training_args = TrainingArguments(
   output_dir =repo_name,
   overwrite_output_dir = True,
   learning_rate=1e-5,
   per_device_train_batch_size=256,
   per_device_eval_batch_size=256,
   num_train_epochs=3,
   weight_decay=0.01,
   evaluation_strategy = 'steps',
   logging_steps = 70,
   eval_steps = 70,
   save_strategy="epoch",
   report_to="wandb"
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
);

In [19]:
trainer.train() #f3deeaa8621234787bf5802550cff771d90288be

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Accuracy,F1
70,0.701900,0.673369,0.554583,0.467097
140,0.654300,0.586112,0.687958,0.686613
210,0.610000,0.571431,0.698292,0.690829
280,0.593100,0.614396,0.674646,0.652998
350,0.579700,0.538877,0.726104,0.725447
420,0.573400,0.541259,0.724708,0.722015
490,0.550900,0.518837,0.740396,0.739602
560,0.547200,0.521442,0.739958,0.737889
630,0.531300,0.504839,0.752917,0.752915
700,0.524400,0.547931,0.719021,0.708796


Step,Training Loss,Validation Loss,Accuracy,F1
70,0.701900,0.673369,0.554583,0.467097
140,0.654300,0.586112,0.687958,0.686613
210,0.610000,0.571431,0.698292,0.690829
280,0.593100,0.614396,0.674646,0.652998
350,0.579700,0.538877,0.726104,0.725447
420,0.573400,0.541259,0.724708,0.722015
490,0.550900,0.518837,0.740396,0.739602
560,0.547200,0.521442,0.739958,0.737889
630,0.531300,0.504839,0.752917,0.752915
700,0.524400,0.547931,0.719021,0.708796


TrainOutput(global_step=2625, training_loss=0.5109440009707497, metrics={'train_runtime': 3073.1016, 'train_samples_per_second': 218.672, 'train_steps_per_second': 0.854, 'total_flos': 1.7266663008e+16, 'train_loss': 0.5109440009707497, 'epoch': 3.0})

In [20]:
trainer.evaluate() 

#  'eval_accuracy': 0.7689583333333333,
#  'eval_f1': 0.7689567384953679,


{'eval_loss': 0.4707125723361969,
 'eval_accuracy': 0.7770625,
 'eval_f1': 0.7770195941827879,
 'eval_runtime': 39.5569,
 'eval_samples_per_second': 1213.441,
 'eval_steps_per_second': 4.753,
 'epoch': 3.0}

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
# model.save_pretrained('diploma_model')

In [27]:
# !mv 'diploma_model' '/content/drive/MyDrive/diploma_model2'

In [23]:
model = AutoModelForSequenceClassification.from_pretrained('diploma_model', num_labels=3, ignore_mismatched_sizes=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at diploma_model and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_path = '/content/drive/MyDrive/diploma_model'
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3, ignore_mismatched_sizes=True)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/diploma_model and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Просто запустим на финансовом без дообучения

In [25]:
repo_name = "just_findf"

training_args = TrainingArguments(
   output_dir =repo_name,
   learning_rate=1e-5,
   per_device_train_batch_size=32,
   per_device_eval_batch_size=32,
   num_train_epochs=30,
   weight_decay=0.01,
   evaluation_strategy = 'steps',
   logging_steps = 20,
   eval_steps = 20,
   save_strategy="epoch",
   report_to="wandb"
)
 
new_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_fin,
    eval_dataset=test_fin,
    compute_metrics=compute_metrics
)
new_trainer.evaluate()

{'eval_loss': 1.0594769716262817,
 'eval_accuracy': 0.5941644562334217,
 'eval_f1': 0.44707722156742186,
 'eval_runtime': 4.2985,
 'eval_samples_per_second': 789.342,
 'eval_steps_per_second': 24.892}

Теперь дообучим только голову на финансовом датасете

In [19]:
model_path = '/content/drive/MyDrive/diploma_model2'
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3, ignore_mismatched_sizes=True)

for param in model.base_model.parameters():
    param.requires_grad = False

repo_name = 'learn_head_findf'
training_args = TrainingArguments(
   output_dir = repo_name,
   overwrite_output_dir = True,
   learning_rate=1e-4,
   per_device_train_batch_size=64,
   per_device_eval_batch_size=64,
   num_train_epochs=100,
   weight_decay=0.01,
   evaluation_strategy = 'steps',
   logging_steps = 20,
   eval_steps = 20,
   save_strategy="epoch",
   report_to="wandb"
)
 
new_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_fin,
    eval_dataset=test_fin,
    compute_metrics=compute_metrics
)

new_trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/diploma_model2 and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Accuracy,F1
20,0.939800,0.928652,0.594164,0.442905
40,0.935600,0.900844,0.593870,0.443460
60,0.909300,0.902169,0.595933,0.448882
80,0.923400,0.903347,0.600648,0.492599
100,0.907700,0.898997,0.599764,0.461082
120,0.905900,0.887522,0.595933,0.448228
140,0.923300,0.923432,0.594459,0.443982
160,0.919500,0.891321,0.600059,0.468060
180,0.886300,0.899215,0.601827,0.469270
200,0.911200,0.897255,0.604775,0.543427


TrainOutput(global_step=2300, training_loss=0.8681766485131305, metrics={'train_runtime': 749.014, 'train_samples_per_second': 193.988, 'train_steps_per_second': 3.071, 'total_flos': 3733435507410000.0, 'train_loss': 0.8681766485131305, 'epoch': 100.0})

In [ ]:
new_trainer.evaluate()

{'eval_loss': 0.8838029503822327,
 'eval_accuracy': 0.596816976127321,
 'eval_f1': 0.4585708803325569,
 'eval_runtime': 3.5981,
 'eval_samples_per_second': 942.985,
 'eval_steps_per_second': 15.008,
 'epoch': 30.0}

In [ ]:
# best_tr = 0
# best_acc = 0
# for tr in np.arange(0, 0.5, 0.05):
#     pred_y = []
#     for elem in pred_ys:
#         if abs(elem[0] - elem[1]) > tr:
#             pred_y.append(np.argmax(elem))
#         else:
#             pred_y.append(2)
#     acc = accuracy_score(pred_y, tokenized_fin['label'])
#     if acc > best_acc:
#         best_acc = acc
#         best_tr = tr

In [ ]:
# print(best_acc, best_tr)

# Предобученная модель

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)

In [21]:
training_args = TrainingArguments(
   output_dir ='kill me',
   learning_rate=5e-5,
   per_device_train_batch_size=32,
   per_device_eval_batch_size=32,
   num_train_epochs=100,
   weight_decay=0.01,
   evaluation_strategy = 'steps',
   logging_steps = 20,
   eval_steps = 20,
   save_strategy="epoch",
   report_to="wandb"
)
 
new_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_fin,
    eval_dataset=test_fin,
    compute_metrics=compute_metrics
)

In [22]:
for param in model.base_model.parameters():
    param.requires_grad = False

new_trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Accuracy,F1
20,1.505800,1.181009,0.358090,0.379367
40,0.968500,0.798123,0.652225,0.544352
60,0.718000,0.610906,0.705865,0.666252
80,0.629300,0.568886,0.747421,0.744389
100,0.607600,0.550397,0.753021,0.747834
120,0.570400,0.538236,0.758621,0.755155
140,0.552800,0.530872,0.764810,0.763216
160,0.546700,0.523685,0.766578,0.761640
180,0.569400,0.517706,0.768641,0.768325
200,0.534500,0.508343,0.773062,0.769593


TrainOutput(global_step=4600, training_loss=0.4307335763392241, metrics={'train_runtime': 1796.5945, 'train_samples_per_second': 80.875, 'train_steps_per_second': 2.56, 'total_flos': 3733435507410000.0, 'train_loss': 0.4307335763392241, 'epoch': 100.0})

In [ ]:
new_trainer.evaluate()

{'eval_loss': 0.4280836284160614,
 'eval_accuracy': 0.8137341585617448,
 'eval_f1': 0.8136185707047193,
 'eval_runtime': 3.8463,
 'eval_samples_per_second': 882.143,
 'eval_steps_per_second': 27.819,
 'epoch': 30.0}

In [ ]:
# model.eval()
# pred_y = []
# pred_ys = []
# with torch.no_grad():
#     for data in tqdm(tokenized_test):
#         ids = torch.tensor([data['input_ids']]).to(device, dtype = torch.long)
#         mask = torch.tensor([data['attention_mask']]).to(device, dtype = torch.long)
#         token_type_ids = torch.tensor([data['token_type_ids']]).to(device, dtype = torch.long)
#         preds = model(ids, mask, token_type_ids)
        
#         pred_y.append(int(preds.logits.argmax(1)[0]))
#         pred_ys.append(preds.logits)

# print(accuracy_score(pred_y, tokenized_test['label']))